# Skeletonization

This notebook aims to apply tools for segmentation and quality measure

In [2]:
import numpy as np 
import kimimaro
import matplotlib.pyplot as plt
import h5py
import neuroglancer
from cloudvolume import CloudVolume
import pandas as pd
from io import StringIO
#import funlib.evaluate
import networkx as nx

from glob import glob

### Load results from ffn segmentation

In [39]:
with np.load('data/fib25_results/fib25/training2/0/0/seg-0_0_0.npz') as data:
    segmentation = data['segmentation']

In [40]:
skels = kimimaro.skeletonize(
  segmentation, 
  teasar_params={
    "scale": 1.5, 
    "const": 300, # physical units
    "pdrf_scale": 100000,
    "pdrf_exponent": 4,
    "soma_acceptance_threshold": 3500, # physical units
    "soma_detection_threshold": 750, # physical units
    "soma_invalidation_const": 300, # physical units
    "soma_invalidation_scale": 2,
    "max_paths": 300, # default None
  },
  # object_ids=[ ... ], # process only the specified labels
  # extra_targets_before=[ (27,33,100), (44,45,46) ], # target points in voxels
  # extra_targets_after=[ (27,33,100), (44,45,46) ], # target points in voxels
  dust_threshold=1000, # skip connected components with fewer than this many voxels
  anisotropy=(10,10,10), # default True
  fix_branching=True, # default True
  fix_borders=True, # default True
  fill_holes=True, # default False
  fix_avocados=True, # default False
  progress=True, # default False, show progress bar
  parallel=1, # <= 0 all cpu, 1 single process, 2+ multiprocess
  parallel_chunk_size=100, # how many skeletons to process before updating progress bar
)


Skeletonizing Labels: 100%|██████████| 251/251 [00:30<00:00,  8.27it/s]


In [41]:
type(skels[1])

cloudvolume.skeleton.Skeleton

### Load ground truth segmentation

In [42]:
with h5py.File('data/fib25_results/flyEM/groundtruth.h5', 'r') as f:
    gt_seg = np.array(f['stack'])
    transforms = np.array(f['transforms'])

In [43]:
gt_seg

array([[[    0,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        ...,
        [   22,    22,    22, ...,    14,    14,    14],
        [   22,    22,    22, ...,    14,    14,    14],
        [   22,    22,    22, ...,    14,    14,    14]],

       [[    0,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        ...,
        [   22,    22,    22, ...,    14,    14,    14],
        [   22,    22,    22, ...,    14,    14,    14],
        [   22,    22,    22, ...,    14,    14,    14]],

       [[    0,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        [    4,     4,     4, ...,     3,     3,     3],
        ...,
        [  165,   165,   165, ...,    14,    14,    14],
        [  165,   165,   165, ...,    14,    

In [44]:
gt_skels = kimimaro.skeletonize(
  gt_seg, 
  teasar_params={
    "scale": 1.5, 
    "const": 300, # physical units
    "pdrf_scale": 100000,
    "pdrf_exponent": 4,
    "soma_acceptance_threshold": 3500, # physical units
    "soma_detection_threshold": 750, # physical units
    "soma_invalidation_const": 300, # physical units
    "soma_invalidation_scale": 2,
    "max_paths": 300, # default None
  },
  # object_ids=[ ... ], # process only the specified labels
  # extra_targets_before=[ (27,33,100), (44,45,46) ], # target points in voxels
  # extra_targets_after=[ (27,33,100), (44,45,46) ], # target points in voxels
  dust_threshold=1000, # skip connected components with fewer than this many voxels
  anisotropy=(10,10,10), # default True
  fix_branching=True, # default True
  fix_borders=True, # default True
  fill_holes=True, # default False
  fix_avocados=True, # default False
  progress=True, # default False, show progress bar
  parallel=1, # <= 0 all cpu, 1 single process, 2+ multiprocess
  parallel_chunk_size=100, # how many skeletons to process before updating progress bar
)

Skeletonizing Labels: 100%|██████████| 2745/2745 [00:15<00:00, 175.82it/s]


### Load raw EM image

In [6]:
with h5py.File('data/fib25_results/flyEM/grayscale_maps.h5', 'r') as f:
    image = np.array(f['raw'])

## Visualise results using neuroglancer (incomplete)

In [15]:
# Initialize Neuroglancer
neuroglancer.set_server_bind_address('127.0.0.1', 8080)
viewer = neuroglancer.Viewer()

In [22]:
info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type='image',
    data_type='uint8',
    encoding='raw',
    resolution=[10, 10, 10],
    voxel_offset=[0, 0, 0],
    chunk_size=[128, 128, 128],
    volume_size=[250, 250, 250],
)

In [23]:
vol = CloudVolume(f'file://data/flyEM/cloudvolume_test', info=info)

In [29]:
CloudVolume.from_numpy(
    segmentation,
    vol_path=f'file://data/flyEM/cloudvolume_test/seg',
    resolution=(10, 10, 10),
    voxel_offset=(0, 0, 0),
    chunk_size=(128, 128, 64),
    layer_type='segmentation',
    encoding='raw',
    max_mip=0
)

In [32]:
CloudVolume.from_numpy(
    image,
    vol_path=f'file://data/flyEM/cloudvolume_test/raw',
    resolution=(10, 10, 10),
    voxel_offset=(0, 0, 0),
    chunk_size=(128, 128, 64),
    layer_type='image',
    encoding='raw',
    max_mip=0,
    compress='br'
)

In [31]:
CloudVolume.from_numpy(
    gt_seg,
    vol_path=f'file://data/flyEM/cloudvolume_test/gt_seg',
    resolution=(10, 10, 10),
    voxel_offset=(0, 0, 0),
    chunk_size=(128, 128, 64),
    layer_type='segmentation',
    encoding='raw',
    max_mip=0
)

In [24]:
vol[:, :, :] = image

Uploading: 100%|██████████| 8/8 [00:00<00:00, 26.35it/s]


## Move skeletons to networkx for quality assessment with funkelab functions

In [40]:
type(skels[1])

cloudvolume.skeleton.Skeleton

In [7]:
# Relevant functions stolen from brainlit

def skel_to_df(skeleton):
    # adapted from brainlit
    swc_string = skeleton.to_swc()
    string_io = StringIO(swc_string)
    splitted_string = swc_string.split("\n")
    in_h = True
    h_len = -1
    while in_h:
        h_len += 1
        line = splitted_string[h_len]
        if len(line) == 0 or line[0] != "#":
            in_h = False
    df = pd.read_table(
        string_io,
        names=["sample", "structure", "x", "y", "z", "r", "parent"],
        skiprows=h_len,
        sep=" "
        # delim_whitespace=True,
    )

    return df


def df_to_graph(df, round=False):
        """Converts dataframe form of neuron trace into a directed graph

        Parameters
        ----------
        df_voxel : :class:`pandas.DataFrame`
            Indices, coordinates, and parents of each node in the swc.
        round : boolean
            Whether coordinates/distances should be rounded to integers.

        Returns
        -------
        G : :class:`networkx.classes.digraph.DiGraph`
            Neuron from swc represented as directed graph. Coordinates x,y,z are
            node attributes accessed by keys 'x','y','z' respectively.
        """
        G = nx.DiGraph()

        # add nodes
        for index, row in df.iterrows():
            id = int(row["sample"])

            coord = [row["x"], row["y"], row["z"]]
            if round:
                coord = [int(c) for c in coord]

            G.add_node(id)
            G.nodes[id]["x"] = coord[0]
            G.nodes[id]["y"] = coord[1]
            G.nodes[id]["z"] = coord[2]

        # add edges
        for index, row in df.iterrows():
            child = int(row["sample"])
            child_coord = [row["x"], row["y"], row["z"]]
            parent = int(row["parent"])

            if parent > min(df["parent"]):
                parent_row = df[df["sample"] == parent]
                parent_coord = [parent_row["x"], parent_row["y"], parent_row["z"]]

                dist = np.linalg.norm(np.subtract(child_coord, parent_coord))
                if round:
                    dist = int(dist)

                G.add_edge(parent, child, distance=dist)

        return G


In [8]:
skelDF = skel_to_df(skels[1])

In [9]:
skelDF

,sample,structure,x,y,z,r,parent
0,1,0,0.0,130.0,1440.0,14.142136,-1
1,61,0,10.0,130.0,1440.0,17.320509,1
2,80,0,20.0,130.0,1440.0,24.494898,61
3,98,0,30.0,130.0,1440.0,33.166248,80
4,121,0,40.0,130.0,1440.0,42.426407,98
...,...,...,...,...,...,...,...
1542,1503,0,1130.0,1960.0,920.0,50.000000,1501
1543,1505,0,1140.0,1950.0,920.0,41.231056,1503
1544,1507,0,1150.0,1940.0,920.0,30.000000,1505
1545,1509,0,1160.0,1930.0,910.0,20.000000,1507


In [10]:
nx_skel = df_to_graph(skelDF)

In [59]:
nx_skel

In [27]:
gt_skels[6137].vertices

array([[ 850.,    0.,   90.],
       [ 850.,   10.,   90.],
       [ 850.,   20.,   90.],
       [ 850.,   30.,   90.],
       [ 850.,   40.,   90.],
       [ 850.,   50.,   90.],
       [ 850.,   60.,   90.],
       [ 850.,   70.,   90.],
       [ 850.,   80.,   90.],
       [ 850.,   90.,   90.],
       [ 850.,  100.,   90.],
       [ 850.,  110.,   90.],
       [ 850.,  120.,   90.],
       [ 850.,  130.,   90.],
       [ 850.,  140.,   90.],
       [ 850.,  150.,   90.],
       [ 850.,  160.,   90.],
       [ 850.,  170.,   90.],
       [ 850.,  180.,   90.],
       [ 850.,  190.,   90.],
       [ 850.,  200.,   90.],
       [ 850.,  210.,   80.],
       [ 850.,  220.,   70.],
       [ 850.,  230.,   60.],
       [ 850.,  240.,   50.],
       [ 850.,  240.,   70.],
       [ 850.,  250.,   40.],
       [ 850.,  250.,   80.],
       [ 850.,  260.,   30.],
       [ 850.,  260.,   90.],
       [ 850.,  270.,   20.],
       [ 850.,  270.,   90.],
       [ 850.,  280.,   10.],
       [ 8

In [34]:
skels[1].vertices

array([[   0.,  130., 1440.],
       [   0.,  190., 1670.],
       [   0.,  200., 1660.],
       ...,
       [1340., 1350., 1830.],
       [1350., 1360., 1840.],
       [1360., 1370., 1850.]], dtype=float32)

In [ ]:
gt_skleDF = skel_to_df(gt_skels[1])

## Expected Run Length calculation

Next steps: implement run length evaluation from https://github.com/funkelab/funlib.evaluate/blob/master/funlib/evaluate/run_length.py or https://github.com/PytorchConnectomics/em_erl. Open questions are matching segmented and ground truth skeletons, incorporating more skeletons

Additional idea from phil: which fraction of neurons would need proofreading, potentially from splits and mergers

In [18]:
# From PytorchConnectomics/em_util

def read_vol_chunk(file_handler, chunk_id=0, chunk_num=1):
    """
    Read a chunk of data from a file handler.

    Args:
        file_handler: The file handler object.
        chunk_id: The ID of the chunk to read. Defaults to 0.
        chunk_num: The total number of chunks. Defaults to 1.

    Returns:
        numpy.ndarray: The read chunk of data.
    """
    if chunk_num == 1:
        # read the whole chunk
        return np.array(file_handler)
    elif chunk_num == -1:
        # read a specific slice
        return np.array(file_handler[chunk_id])
    else:
        # read a chunk
        num_z = int(np.ceil(file_handler.shape[0] / float(chunk_num)))
        return np.array(file_handler[chunk_id * num_z : (chunk_id + 1) * num_z])

with h5py.File('data/em_erl/test_50_skeletons.h5', 'r') as f:
    foo = list(f)
    out = [None] * len(foo)
    for di, d in enumerate(foo):
        out[di] = read_vol_chunk(f[d], 0, 1)


### Create look up table comparing skeleton to segmentations

In [ ]:
# Stolen from PytorchConnectomics/em_erl
# https://github.com/PytorchConnectomics/em_erl/blob/main/em_erl/eval.py

def compute_segment_lut(
    segment, node_position, mask=None, chunk_num=1, data_type=np.uint32
):
    """
    The function `compute_segment_lut` is a low memory version of a lookup table
    computation for node segments in a 3D volume.

    :param node_position: A numpy array containing the coordinates of each node. The shape of the array
    is (N, 3), where N is the number of nodes and each row represents the (z, y, x) coordinates of a
    node
    :param segment: either a 3D volume or a string representing the
    name of a file containing segment data.
    :param chunk_num: The parameter `chunk_num` is the number of chunks into which the volume is divided
    for reading. It is used in the `read_vol` function to specify which chunk to read, defaults to 1
    (optional)
    :param data_type: The parameter `data_type` is the data type of the array used to store the node segment
    lookup table. In this case, it is set to `np.uint32`, which means the array will store unsigned
    32-bit integers
    :return: a list of numpy arrays, where each array represents the node segment lookup table for a
    specific segment.
    """
    if not isinstance(segment, str):
        # load the whole segment
        node_lut = segment[
            node_position[:, 0], node_position[:, 1], node_position[:, 2]
        ]
        mask_id = []
        if mask is not None:
            if isinstance(mask, str):
                mask = read_vol(mask)
            mask_id = segment[mask > 0]
    else:
        # read segment by chunk (when memory is limited)
        assert ".h5" in segment
        node_lut = np.zeros(node_position.shape[0], data_type)
        mask_id = [[]] * chunk_num
        start_z = 0
        for chunk_id in range(chunk_num):
            seg = read_vol(segment, None, chunk_id, chunk_num)
            last_z = start_z + seg.shape[0]
            ind = (node_position[:, 0] >= start_z) * (node_position[:, 0] < last_z)
            pts = node_position[ind]
            node_lut[ind] = seg[pts[:, 0] - start_z, pts[:, 1], pts[:, 2]]
            if mask is not None:
                if isinstance(mask, str):
                    mask_z = read_vol(mask, None, chunk_id, chunk_num)
                else:
                    mask_z = mask[start_z:last_z]
                mask_id[chunk_id] = seg[mask_z > 0]
            start_z = last_z
        if mask is not None:
            # remove irrelevant seg ids (not used by nodes)
            node_lut_unique = np.unique(node_lut)
            for chunk_id in range(chunk_num):
                mask_id[chunk_id] = mask_id[chunk_id][
                    np.in1d(mask_id[chunk_id], node_lut_unique)
                ]
        mask_id = np.concatenate(mask_id)
    return node_lut, mask_id

In [46]:
lut = compute_segment_lut(gt_seg, (skels[1].vertices/10).astype(int))

In [58]:
lut[0]

array([   38,    38,    38, ..., 11239, 11362, 11484], dtype=int32)

In [50]:
gt_lut = compute_segment_lut(gt_seg, (gt_skels[6137].vertices/10).astype(int))

In [51]:
gt_lut

(array([6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137, 6137,
        6137, 6137, 6137, 6137, 6137, 

code works, gt_lut is accurate, segmentation lut shows errors in segmentation

In [98]:
skels

{1: Skeleton(segid=1, vertices=(shape=1547, float32), edges=(shape=1546, uint32), radius=(1547, float32), vertex_types=(1547, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 91: Skeleton(segid=91, vertices=(shape=43, float32), edges=(shape=42, uint32), radius=(43, float32), vertex_types=(43, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 145: Skeleton(segid=145, vertices=(shape=57, float32), edges=(shape=56, uint32), radius=(57, float32), vertex_types=(57, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 117: Skeleton(segid=117, vertices=(shape=284, float32), edges=(shape=283, uint32), radius=(284, float32), vertex_types=(284, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 4: Skeleton(segid=4, vertices=(shape=1071, float32), edges=(shape=1070, uint32), radius=(1071, float3

In [104]:
skels.keys()

dict_keys([1, 91, 145, 117, 4, 126, 14, 7, 224, 69, 8, 9, 227, 205, 123, 24, 85, 152, 96, 197, 175, 103, 156, 206, 186, 148, 182, 213, 11, 167, 178, 207, 253, 173, 129, 139, 116, 232, 185, 150, 79, 41, 93, 217, 143, 25, 115, 142, 162, 92, 159, 33, 239, 22, 170, 125, 157, 240, 81, 46, 176, 110, 31, 27, 75, 121, 106, 202, 76, 112, 16, 19, 35, 172, 235, 149, 21, 71, 204, 95, 122, 32, 153, 54, 63, 252, 111, 80, 216, 50, 13, 99, 60, 124, 57, 223, 47, 74, 219, 138, 177, 42, 134, 48, 147, 58, 161, 118, 39, 230, 83, 100, 34, 43, 56, 87, 23, 238, 211, 131, 228, 154, 198, 163, 140, 171, 59, 237, 109, 89, 10, 38, 221, 66, 65, 113, 218, 233, 77, 26, 146, 127, 241, 231, 132, 180, 12, 160, 6, 44, 90, 247, 45, 256, 209, 20, 108, 52, 101, 40, 86, 201, 36, 155, 193, 248, 105, 203, 244, 212, 168, 174, 158, 29, 70, 245, 107, 49, 82, 192, 78, 189, 226, 72, 254, 30, 51, 98, 64, 208, 214, 61, 199, 15, 55, 234, 165, 190, 236, 195, 179, 68, 5, 215, 144, 255, 251, 17, 3, 210, 250, 225, 2, 222, 141, 200, 191, 2

In [127]:
# for all graphs
seg_luts = []
seg_preds = np.zeros((len(skels), 2))
for i, key in enumerate(skels.keys()):
    skel = skels[key]
    node_lut, mask_id = compute_segment_lut(gt_seg, (skel.vertices/10).astype(int))
    seg_luts.append(node_lut)

    values, counts = np.unique(node_lut, return_counts=True)
    ind = np.argmax(counts)
    seg_preds[i, 0] = key
    seg_preds[i, 1] = values[ind]

seg_preds = seg_preds.astype(int)


In [130]:
seg_luts[83]

array([3469, 3469, 3534, 3534, 3534, 3534, 3534, 3534, 3600, 3600, 3600,
       3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3613, 3613,
       3669, 3669, 3669, 3669, 3669, 3669, 3742], dtype=int32)

### Get erl score from em_erl

In [47]:
# Works due to changes in .py files directly (from em_util.io import seg_relabel -> from em_util.em_util.io import seg_relabel)
# and because em_util in same folder
from em_erl.eval import compute_erl_score
from em_erl.erl import skel_to_erlgraph

In [48]:
# get prediction for which gt segmentation belongs to some skeleton
#skel_num, pred_seg_num = seg_preds[83, :]

# get gt graph from gt skeleton
gt_graph = skel_to_erlgraph(gt_skels)

# compute erl score
#compute_erl_score(gt_graph, seg_luts[83], [], 0, verbose=True)

In [49]:
pred_graph = skel_to_erlgraph(skels)

In [22]:
node_position = pred_graph.get_nodes_position((10,10,10))

In [25]:
pred_graph.get_nodes_position().shape

(26380, 3)

In [27]:
node_position

array([[  0,  13, 144],
       [  0,  19, 167],
       [  0,  20, 166],
       ...,
       [117, 131, 233],
       [117, 134, 235],
       [117, 135, 235]])

In [155]:
node_segment_lut, mask_segment_id = compute_segment_lut(gt_seg, node_position, None)

In [156]:
score = compute_erl_score(erl_graph=pred_graph,
    node_segment_lut=node_segment_lut,
    mask_segment_id=mask_segment_id,
    merge_threshold=0,
    verbose=True)

In [157]:
score.compute_erl(None)

In [158]:
score.print_erl()

all skel
ERL	: 1.63
gt ERL	: 7539.24
#skel	: 249
-----------------


In [151]:
gt_node_segment_lut, gt_mask_segment_id = compute_segment_lut(gt_seg, node_position, None)
gt_score = compute_erl_score(erl_graph=gt_graph,
    node_segment_lut=gt_node_segment_lut,
    mask_segment_id=gt_mask_segment_id,
    merge_threshold=0,
    verbose=True)

gt_score.compute_erl(None)
gt_score.print_erl()

all skel
ERL	: 5709.76
gt ERL	: 5709.76
#skel	: 2327
-----------------


as we can see, erl is computed but shows that segmentation is trash. This doesn't fit expectations as visually, segmentation seemed ok. My guess is that there is an issue in the data format for ground truth segmentation: When examining data in playground.ipynb, slicing across z gives good results for both prediction and gt, but slicing through y or x gives only good results for prediction. In some way, ground truth segmentation is not aligned with raw image.

In [43]:
fnames = glob('data/microns/em/' + '*.npy')

In [1]:
def load_data(path):
    """
    Helper function to load all npy and npz files in a directory into a single numpy array.
    Requires all files to have the same shape.

    Parameters
    ----------
    path : str
        The path to the directory containing the npy and npz files.

    Returns
    -------
    np.ndarray
        A numpy array containing the loaded data.
    """
    files = sorted(glob(path + "*.npy"))

    data_shape = np.load(files[0]).shape

    out = np.zeros((data_shape[0], data_shape[1], len(files)))

    for i, f in enumerate(files):
        out[:, :, i] = np.load(f)

    return out.astype(int)

In [3]:
gt_seg = load_data("data/microns/seg_v1078/")

In [4]:
gt_seg = gt_seg - (np.min(gt_seg[np.nonzero(gt_seg)]) - 1)

In [5]:
gt_seg = gt_seg.astype(np.uint32)

In [ ]:
gt_skels = kimimaro.skeletonize(
        gt_seg, 
        teasar_params={
            "scale": 1.5, 
            "const": 300, # physical units
            "pdrf_scale": 100000,
            "pdrf_exponent": 4,
            "soma_acceptance_threshold": 3500, # physical units
            "soma_detection_threshold": 750, # physical units
            "soma_invalidation_const": 300, # physical units
            "soma_invalidation_scale": 2,
            "max_paths": 300, # default None
        },
        dust_threshold=1000, # skip connected components with fewer than this many voxels
        anisotropy=(4,4,40), # default True
        fix_branching=True, # default True
        fix_borders=True, # default True
        fill_holes=True, # default False
        fix_avocados=True, # default False
        progress=True, # default False, show progress bar
        parallel=1, # <= 0 all cpu, 1 single process, 2+ multiprocess
        parallel_chunk_size=100, # how many skeletons to process before updating progress bar
        )

In [6]:
proof_seg = load_data('data/microns/seg_v117/')

In [7]:
proof_seg = proof_seg - (np.min(proof_seg[np.nonzero(proof_seg)]) - 1)
proof_seg = proof_seg.astype(np.uint32)

In [8]:
proof_skels = kimimaro.skeletonize(
        proof_seg, 
        teasar_params={
            "scale": 1.5, 
            "const": 300, # physical units
            "pdrf_scale": 100000,
            "pdrf_exponent": 4,
            "soma_acceptance_threshold": 3500, # physical units
            "soma_detection_threshold": 750, # physical units
            "soma_invalidation_const": 300, # physical units
            "soma_invalidation_scale": 2,
            "max_paths": 300, # default None
        },
        dust_threshold=1000, # skip connected components with fewer than this many voxels
        anisotropy=(4, 4, 40), # default True
        fix_branching=True, # default True
        fix_borders=True, # default True
        fill_holes=True, # default False
        fix_avocados=True, # default False
        progress=True, # default False, show progress bar
        parallel=1, # <= 0 all cpu, 1 single process, 2+ multiprocess
        parallel_chunk_size=100, # how many skeletons to process before updating progress bar
        )

Skeletonizing Labels: 100%|██████████| 602/602 [00:58<00:00, 10.30it/s]


In [9]:
from zmesh import Mesher


mesher = Mesher((4,4,40))

#mesher.mesh(gt_seg, close=True)

In [10]:
proof_mesher = Mesher((4,4,40))

proof_mesher.mesh(proof_seg, close=True)

proof_meshes = dict()

for obj_id in proof_mesher.ids():
  mesh = proof_mesher.get(
      obj_id, 
      normals=False, # whether to calculate normals or not

      # tries to reduce triangles by this factor
      # 0 disables simplification
      reduction_factor=0, 

      # Max tolerable error in physical distance
      # note: if max_error is not set, the max error
      # will be set equivalent to one voxel along the 
      # smallest dimension.
      max_error=8,
      # whether meshes should be centered in the voxel
      # on (0,0,0) [False] or (0.5,0.5,0.5) [True]
      voxel_centered=False, 
    )
  proof_meshes[obj_id] = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
  mesher.erase(obj_id) # delete high res mesh
  del mesh

NameError: name 'trimesh' is not defined

In [42]:
proof_meshes

{2740831361: <trimesh.Trimesh(vertices.shape=(144, 3), faces.shape=(284, 3))>,
 2826723201: <trimesh.Trimesh(vertices.shape=(238, 3), faces.shape=(472, 3))>,
 3070268161: <trimesh.Trimesh(vertices.shape=(34, 3), faces.shape=(64, 3))>,
 2740798081: <trimesh.Trimesh(vertices.shape=(164, 3), faces.shape=(324, 3))>,
 3741563137: <trimesh.Trimesh(vertices.shape=(246, 3), faces.shape=(488, 3))>,
 2740834433: <trimesh.Trimesh(vertices.shape=(1348, 3), faces.shape=(2692, 3))>,
 995844737: <trimesh.Trimesh(vertices.shape=(182, 3), faces.shape=(360, 3))>,
 2695847553: <trimesh.Trimesh(vertices.shape=(488, 3), faces.shape=(960, 3))>,
 1614159617: <trimesh.Trimesh(vertices.shape=(438, 3), faces.shape=(872, 3))>,
 3108927361: <trimesh.Trimesh(vertices.shape=(3150, 3), faces.shape=(6296, 3))>,
 227864577: <trimesh.Trimesh(vertices.shape=(410, 3), faces.shape=(816, 3))>,
 671942913: <trimesh.Trimesh(vertices.shape=(1548, 3), faces.shape=(3092, 3))>,
 3226380929: <trimesh.Trimesh(vertices.shape=(752, 

In [43]:
proof_skels

{3403396225: Skeleton(segid=3403396225, vertices=(shape=380, float32), edges=(shape=379, uint32), radius=(380, float32), vertex_types=(380, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 3083699841: Skeleton(segid=3083699841, vertices=(shape=52, float32), edges=(shape=51, uint32), radius=(52, float32), vertex_types=(52, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 1620744833: Skeleton(segid=1620744833, vertices=(shape=173, float32), edges=(shape=172, uint32), radius=(173, float32), vertex_types=(173, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 3395813633: Skeleton(segid=3395813633, vertices=(shape=642, float32), edges=(shape=641, uint32), radius=(642, float32), vertex_types=(642, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 251771137: Skeleton(segid=251771137, ve

In [44]:
import skeletor as sk

proof_skeletor_skels = dict()
for id in proof_meshes:
    mesh = proof_meshes[id]
    mesh = sk.pre.fix_mesh(mesh, remove_disconnected=5, inplace=False)
    skel = sk.skeletonize.by_wavefront(mesh, waves=1, step_size=1)
    proof_skeletor_skels[id] = skel
    del mesh
    del skel

Skeletonizing:   0%|          | 0/144 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/238 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/34 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/164 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/246 [00:00<?, ?it/s]

...

Skeletonizing:   0%|          | 0/336 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/68016 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/8996 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/12786 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/45378 [00:00<?, ?it/s]

In [13]:
meshes = dict()
for obj_id in mesher.ids():
  meshes[obj_id] = mesher.get(
      obj_id, 
      normals=False, # whether to calculate normals or not

      # tries to reduce triangles by this factor
      # 0 disables simplification
      reduction_factor=0, 

      # Max tolerable error in physical distance
      # note: if max_error is not set, the max error
      # will be set equivalent to one voxel along the 
      # smallest dimension.
      max_error=8,
      # whether meshes should be centered in the voxel
      # on (0,0,0) [False] or (0.5,0.5,0.5) [True]
      voxel_centered=False, 
    )
  mesher.erase(obj_id) # delete high res mesh

In [14]:
meshes.keys()

dict_keys([3703648513, 3595279873, 2740798081, 3071324929, 995844737, 2695847553, 1614159617, 3108927361, 4266257025, 227864577, 671942913, 22909057, 536177537, 216910721, 428049025, 22918529, 22916737, 428047745, 995854721, 22909313, 3225796993, 3131400065, 230584321, 216918401, 245096193, 216905857, 3673933313, 295004417, 428048513, 22917505, 245100801, 241282305, 428047233, 1618142721, 2976548225, 3424646017, 22908289, 3405221249, 216913025, 1173769217, 428048257, 3510992129, 245105153, 1978282497, 216917121, 1978283009, 995843713, 174113537, 3545637633, 216907137, 3376752769, 1614161153, 4009269121, 2695847297, 1978279937, 3261520257, 22919809, 245096705, 174112769, 2740805761, 1173768705, 995845505, 3226191489, 3257735041, 174111745, 428044417, 3309538305, 3699187201, 3257721217, 2695843713, 22910081, 22919553, 219733505, 245106689, 22912129, 1978284033, 1978283265, 2740799105, 174114305, 3674070529, 3131483009, 1614160641, 3563050881, 174115073, 536175233, 22906753, 3405339777, 2

In [15]:
import trimesh

trimeshes = dict()

for mesh_key in meshes:
    trimeshes[mesh_key] = trimesh.Trimesh(meshes[mesh_key].vertices, meshes[mesh_key].faces)

In [16]:
def compute_mesh_lut(
    meshes, node_position, anisotropy=(4, 4, 40), data_type=np.uint32
):
    """
    """

    scaling_factor = (1/anisotropy[0], 1/anisotropy[1], 1/anisotropy[2])

    node_lut = np.zeros(node_position.shape[0], dtype=data_type)
    for mesh_id in meshes:
        mesh = meshes[mesh_id]
        mesh.apply_scale(scaling_factor)

        contains = mesh.contains(node_position)
        node_lut[contains] = mesh_id

    return node_lut

In [17]:
from em_erl.erl import skel_to_erlgraph
from em_erl.eval import compute_erl_score

In [18]:
# Stolen from PytorchConnectomics/em_erl
# https://github.com/PytorchConnectomics/em_erl/blob/main/em_erl/eval.py
def compute_segment_lut(
    segment, node_position, mask=None, chunk_num=1, data_type=np.uint32
):
    """
    The function `compute_segment_lut` is a low memory version of a lookup table
    computation for node segments in a 3D volume.

    :param node_position: A numpy array containing the coordinates of each node. The shape of the array
    is (N, 3), where N is the number of nodes and each row represents the (z, y, x) coordinates of a
    node
    :param segment: either a 3D volume or a string representing the
    name of a file containing segment data.
    :param chunk_num: The parameter `chunk_num` is the number of chunks into which the volume is divided
    for reading. It is used in the `read_vol` function to specify which chunk to read, defaults to 1
    (optional)
    :param data_type: The parameter `data_type` is the data type of the array used to store the node segment
    lookup table. In this case, it is set to `np.uint32`, which means the array will store unsigned
    32-bit integers
    :return: a list of numpy arrays, where each array represents the node segment lookup table for a
    specific segment.
    """
    if not isinstance(segment, str):
        # load the whole segment
        node_lut = segment[
            node_position[:, 0], node_position[:, 1], node_position[:, 2]
        ]
        mask_id = []
        if mask is not None:
            if isinstance(mask, str):
                mask = read_vol(mask)
            mask_id = segment[mask > 0]
    else:
        # read segment by chunk (when memory is limited)
        assert ".h5" in segment
        node_lut = np.zeros(node_position.shape[0], data_type)
        mask_id = [[]] * chunk_num
        start_z = 0
        for chunk_id in range(chunk_num):
            seg = read_vol(segment, None, chunk_id, chunk_num)
            last_z = start_z + seg.shape[0]
            ind = (node_position[:, 0] >= start_z) * (node_position[:, 0] < last_z)
            pts = node_position[ind]
            node_lut[ind] = seg[pts[:, 0] - start_z, pts[:, 1], pts[:, 2]]
            if mask is not None:
                if isinstance(mask, str):
                    mask_z = read_vol(mask, None, chunk_id, chunk_num)
                else:
                    mask_z = mask[start_z:last_z]
                mask_id[chunk_id] = seg[mask_z > 0]
            start_z = last_z
        if mask is not None:
            # remove irrelevant seg ids (not used by nodes)
            node_lut_unique = np.unique(node_lut)
            for chunk_id in range(chunk_num):
                mask_id[chunk_id] = mask_id[chunk_id][
                    np.in1d(mask_id[chunk_id], node_lut_unique)
                ]
        mask_id = np.concatenate(mask_id)
    return node_lut, mask_id

In [19]:
gt_graph = skel_to_erlgraph(gt_skels)
proof_graph = skel_to_erlgraph(proof_skels)

In [20]:
nodes_position = proof_graph.get_nodes_position((4, 4, 40))

node_segment_lut, mask_segment_id = compute_segment_lut(gt_seg, nodes_position)

In [21]:
mask_segment_id

[]

In [29]:
score = compute_erl_score(erl_graph=gt_graph,
    node_segment_lut=node_segment_lut,
    mask_segment_id=mask_segment_id,
    merge_threshold=0,
    verbose=True)

In [30]:
score.compute_erl(None)
score.print_erl()

all skel
ERL	: 8675.36
gt ERL	: 12043.16
#skel	: 484
-----------------


In [31]:
node_segment_lut

array([3403396225, 3403396225, 3403396225, ..., 3186486657, 3186486657,
       3186486657], dtype=uint32)

In [22]:
mesh_lut = compute_mesh_lut(trimeshes, nodes_position)

In [23]:
mesh_lut

array([         0,          0,          0, ...,  671937281, 3186486657,
       3186486657], dtype=uint32)

In [24]:
nodes_position[mesh_lut != node_segment_lut]

array([[  0,   1,   0],
       [  0,   4,   5],
       [  0,   5,   5],
       ...,
       [251, 164, 458],
       [252, 165, 458],
       [253, 166, 458]])

In [27]:
node_segment_lut[mesh_lut != node_segment_lut]

array([3403396225, 3403396225, 3403396225, ..., 3186486657, 3186486657,
       3186486657], dtype=uint32)

In [30]:
uniques, counts = np.unique(mesh_lut[mesh_lut != node_segment_lut], return_counts=True)

In [35]:
uniques[np.argsort(counts)]

array([ 295004417,  216910721,  216912257,  230489857, 3225797249,
       3673933313,  241334273,  281040641,  281042433, 3403449985,
       3131483009,  281052417,  281053697,  329234433, 3403442049,
       3904344193,  339267201,  400397185, 3257620865,  536176001,
       3088692097, 3257700737, 3384355841, 1444150657, 3379770369,
       1614160641, 2970860289, 3376667521, 1978283265, 1978348801,
       3360152193,  339270529, 3990368129, 3304855937,       1537,
        174109185,   22909569,  174106625,     111617,  174115073,
           107777,  182641025,  174109697,       2305,  281042689,
        281053441, 2919353345, 3376709249,  120895361, 3163806593,
       3403429249, 3403396225,  281042177, 3257590145, 3403394177,
       1978348545, 3372112257,  848383617,  995854721, 3390133377,
       1444149121,       7681, 1614148865, 3376663937, 4205571329,
        423442177,  281035777, 4009269121,  245106689, 3499102465,
       2695852417, 3303521665,  232611329,  232611585, 3132404

In [36]:
trimeshes[3403396225].vertices

TrackedArray([[ 68. , 236.5,  40. ],
              [ 68. , 236. ,  40.5],
              [ 68.5, 236. ,  40. ],
              ...,
              [  0.5,   1. ,   6. ],
              [  0.5,   1. ,   5. ],
              [  0.5,   1. ,   1. ]])

In [63]:
trimeshes[3403396225].contains(np.array([[0, 1, 0]]))

array([False])

In [33]:
(mesh_lut == node_segment_lut).sum()

93051

In [64]:
(mesh_lut == node_segment_lut).sum()/mesh_lut.size

0.790538718346926

In [34]:
score = compute_erl_score(erl_graph=gt_graph,
    node_segment_lut=mesh_lut,
    mask_segment_id=mask_segment_id,
    merge_threshold=0,
    verbose=True)

score.compute_erl(None)
score.print_erl()

all skel
ERL	: 3.07
gt ERL	: 12043.16
#skel	: 484
-----------------


In [90]:
gt_graph.skeleton_id

array([864691135589335552, 864691135269639168, 864691133806684160,
       864691136106568448, 864691136732677760, 864691135349709568,
       864691135976860032, 864691135571476992, 864691135531898368,
       864691136601801984, 864691132402843904, 864691135411893760,
       864691135539138688, 864691134881786624, 864691135411358976,
       864691135411497216, 864691135589381376, 864691135700706048,
       864691135349710848, 864691135443660544, 864691135589368576,
       864691136722192512, 864691134164295040, 864691135351569152,
       864691135562606848, 864691136617699840, 864691136488414976,
       864691136813212288, 864691135349746944, 864691135582673024,
       864691135932862336, 864691135317353984, 864691135385703552,
       864691135778324864, 864691133806688768, 864691136722238848,
       864691136176307456, 864691135518448512, 864691132430975872,
       864691135183101696, 864691135359918720, 864691135412775168,
       864691134881791744, 864691135490795264, 864691135349739

In [83]:
score = compute_erl_score(erl_graph=gt_graph,
    node_segment_lut=mesh_lut,
    mask_segment_id=mask_segment_id,
    merge_threshold=0,
    verbose=True)

In [84]:
score.compute_erl(None)
score.print_erl()

all skel
ERL	: 3.19
gt ERL	: 10822.92
#skel	: 483
-----------------


In [14]:
skeleton_segment, count = np.unique(
        np.hstack([gt_graph.nodes[:, :1], node_segment_lut.reshape(-1, 1)]),
        axis=0,
        return_counts=True,
    )

In [15]:
segments, num_segment_skeletons = np.unique(
        skeleton_segment[:, 1], return_counts=True
    )

In [16]:
merged_segments = segments[num_segment_skeletons > 1]
merged_skeletons = np.unique(
    skeleton_segment[np.isin(skeleton_segment[:, 1], merged_segments), 0]
)

In [82]:
len(merged_segments)

94

In [63]:
# Funkelab implementation


merges = {}
merging_segments = segments[num_segment_skeletons > 1]
merging_segments_mask = np.isin(skeleton_segment[:, 1], merging_segments)
merged_segments = skeleton_segment[:, 1][merging_segments_mask]

merged_skeletons = skeleton_segment[:, 0][merging_segments_mask]

for segment, skeleton in zip(merged_segments, merged_skeletons):
    if segment not in merges:
        merges[segment] = []
    merges[segment].append(skeleton)

In [64]:
merges

{864691135279377536: [94],
 864691135455287936: [95, 96],
 864691135589392640: [97, 98],
 864691136721531520: [99, 100, 102],
 864691135210806272: [101, 103],
 864691136711407744: [104, 106],
 864691135443679744: [105, 107],
 864691135589366784: [108, 110],
 864691135443675904: [109, 111, 112],
 864691135777319808: [113, 114],
 864691136135686528: [115, 116, 118],
 864691135443668224: [117, 119],
 864691135450556672: [120, 122, 124],
 864691133630085120: [121, 123],
 864691136713664896: [125, 127],
 864691135885126528: [126, 129],
 864691135576072704: [128, 130, 132],
 864691135412130816: [131, 133],
 864691136237486848: [134, 136],
 864691135066026624: [135, 137, 138],
 864691135349745920: [139, 140],
 864691135411736576: [141, 142, 144],
 864691135348018944: [143, 145, 146, 147],
 864691135589374720: [148, 149, 150],
 864691135947195392: [151, 152, 153, 154],
 864691136452249088: [155, 157],
 864691133800088704: [156, 158, 159],
 864691135416568192: [160, 162],
 864691133630085632: [

In [65]:
skeletons = gt_skels[864691135589335552]

In [73]:
skeletons

Skeleton(segid=864691135589335552, vertices=(shape=380, float32), edges=(shape=379, uint32), radius=(380, float32), vertex_types=(380, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

In [66]:
skeletons.vertices

array([[   0.,    4.,    0.],
       [   0.,   16.,  200.],
       [   0.,   20.,  200.],
       ...,
       [ 260.,  936., 1560.],
       [ 264.,  936., 1560.],
       [ 268.,  940., 1560.]], dtype=float32)

In [71]:
skeletons.edges

array([[  0,  52],
       [  1,   2],
       [  1,  54],
       [  2,  55],
       [  3,  57],
       [  4,  58],
       [  5,  59],
       [  6,  60],
       [  7,  61],
       [  8,   9],
       [  8,  62],
       [  9,  10],
       [ 10,  11],
       [ 11,  12],
       [ 12,  13],
       [ 13,  14],
       [ 14,  15],
       [ 15,  16],
       [ 16,  17],
       [ 17,  18],
       [ 18,  19],
       [ 19,  20],
       [ 20,  21],
       [ 21,  22],
       [ 22,  23],
       [ 23,  24],
       [ 24,  25],
       [ 25,  26],
       [ 26,  27],
       [ 27,  28],
       [ 28,  29],
       [ 29,  30],
       [ 30,  31],
       [ 31,  32],
       [ 32,  33],
       [ 33,  34],
       [ 34,  35],
       [ 35,  36],
       [ 36,  37],
       [ 37,  38],
       [ 38,  39],
       [ 39,  40],
       [ 40,  41],
       [ 41,  42],
       [ 42,  43],
       [ 43,  44],
       [ 44,  45],
       [ 45,  46],
       [ 46,  47],
       [ 47,  48],
       [ 48,  49],
       [ 49,  50],
       [ 50,

In [68]:
node_segment_lut[0]

864691135589335552

In [69]:
node_segment_lut[52]

864691135589335552

In [72]:
for u, v in skeletons.edges():

    skeleton_id = skeletons.nodes[u][skeleton_id_attribute]
    segment_u = node_segment_lut[u]
    segment_v = node_segment_lut[v]

AttributeError: 'Skeleton' object has no attribute 'nodes'

In [ ]:
splits = {}
n_splits = 0
for skeleton_id in proof_skels:
    
    skeletons = proof_skels[skeleton_id]
    
    for u, v in skeletons.edges:
        segment_u = node_segment_lut[u]
        segment_v = node_segment_lut[v]

    if segment_u != segment_v:
        n_splits += 1

        if skeleton_id not in splits:
            splits[skeleton_id] = []
        splits[skeleton_id].append((segment_u, segment_v))

### Count Mergers and Splits, return number of problematic skeletons

In [83]:
problem_segments = []
n_merges = 0
n_splits = 0


# Mergers
# TODO: why different number of mergers than funkelab implementation?
skeleton_segment, count = np.unique(
        np.hstack([gt_graph.nodes[:, :1], node_segment_lut.reshape(-1, 1)]),
        axis=0,
        return_counts=True,
    )
segments, num_segment_skeletons = np.unique(
        skeleton_segment[:, 1], return_counts=True
    )

merged_segments = segments[num_segment_skeletons > 1]
merging_segments_mask = np.isin(skeleton_segment[:, 1], merged_segments)

merged_segments = skeleton_segment[:, 1][merging_segments_mask]

merged_skeletons = np.unique(
    skeleton_segment[np.isin(skeleton_segment[:, 1], merged_segments), 0]
)

n_mergers = len(merged_segments)

problem_segments += list(merged_segments)



# Splits TODO: probably inefficient
for skeleton_id in proof_skels:
    
    skeletons = proof_skels[skeleton_id]
    
    for u, v in skeletons.edges:
        segment_u = node_segment_lut[u]
        segment_v = node_segment_lut[v]

    if segment_u != segment_v:
        n_splits += 1

        if skeleton_id not in problem_segments:
            problem_segments += [skeleton_id]


In [84]:
n_mergers

248

In [77]:
len(merges)

59

In [75]:
len(problem_segments)

96

In [77]:
np.unique(merged_segments, return_counts=True)

(array([864691132515172736, 864691132525210368, 864691133034133760,
        864691133034323456, 864691133034324480, 864691133630085120,
        864691133630085632, 864691133800088704, 864691133800124288,
        864691133800124544, 864691133806662144, 864691134164279168,
        864691134881784064, 864691134881790976, 864691134941157760,
        864691134941329024, 864691135013990144, 864691135066026624,
        864691135210806272, 864691135279377536, 864691135281401088,
        864691135348018944, 864691135349532160, 864691135349578752,
        864691135349709824, 864691135349718784, 864691135349745920,
        864691135349757696, 864691135356964864, 864691135357842688,
        864691135368178688, 864691135411736576, 864691135412130816,
        864691135416568192, 864691135443529728, 864691135443529984,
        864691135443668224, 864691135443669760, 864691135443670272,
        864691135443675904, 864691135443679744, 864691135450556672,
        864691135455287936, 864691135463478400, 

In [83]:
len(np.unique(merged_segments))

94

In [80]:
merges

{864691135279377536: [94],
 864691135455287936: [95, 96],
 864691135589392640: [97, 98],
 864691136721531520: [99, 100, 102],
 864691135210806272: [101, 103],
 864691136711407744: [104, 106],
 864691135443679744: [105, 107],
 864691135589366784: [108, 110],
 864691135443675904: [109, 111, 112],
 864691135777319808: [113, 114],
 864691136135686528: [115, 116, 118],
 864691135443668224: [117, 119],
 864691135450556672: [120, 122, 124],
 864691133630085120: [121, 123],
 864691136713664896: [125, 127],
 864691135885126528: [126, 129],
 864691135576072704: [128, 130, 132],
 864691135412130816: [131, 133],
 864691136237486848: [134, 136],
 864691135066026624: [135, 137, 138],
 864691135349745920: [139, 140],
 864691135411736576: [141, 142, 144],
 864691135348018944: [143, 145, 146, 147],
 864691135589374720: [148, 149, 150],
 864691135947195392: [151, 152, 153, 154],
 864691136452249088: [155, 157],
 864691133800088704: [156, 158, 159],
 864691135416568192: [160, 162],
 864691133630085632: [

## Starting from Meshes

In [11]:
from meshparty import trimesh_io, skeletonize
from trimesh.exchange.off import load_off
from trimesh import Trimesh

In [13]:
with open('data/neurd/864691135526405723_dynamic.off', 'r') as f:
    mesh_params = load_off(f)

mesh_dynamic = Trimesh(vertices=mesh_params["vertices"], faces=mesh_params["faces"])

In [29]:
mesh_dynamic.faces

TrackedArray([[1765308, 1763160, 1765309],
              [1765309, 1763160, 1764159],
              [1758286, 1759354, 1759894],
              ...,
              [ 359930,  359235,  360186],
              [ 359235,  359932,  360186],
              [ 360472,  360186,  359932]])

In [31]:
mesh_dynamic = trimesh_io.Mesh(vertices=mesh_dynamic.vertices, faces=mesh_dynamic.faces)

In [33]:
skel_dynamic = skeletonize.skeletonize_mesh(mesh_dynamic)

/home/nicolas/Desktop/projects/eon_systems/proofreading/.conda/lib/python3.11/site-packages/meshparty/skeletonize.py:622: RuntimeWarning: invalid value encountered in multiply
  target = np.nanargmax(root_ds * valid)


In [34]:
with open('data/neurd/864691135526405723_static.off', 'r') as f:
    mesh_params = load_off(f)

mesh_static = trimesh_io.Mesh(vertices=mesh_params["vertices"], faces=mesh_params["faces"])

skel_static = skeletonize.skeletonize_mesh(mesh_static)

/home/nicolas/Desktop/projects/eon_systems/proofreading/.conda/lib/python3.11/site-packages/meshparty/skeletonize.py:622: RuntimeWarning: invalid value encountered in multiply
  target = np.nanargmax(root_ds * valid)


In [36]:
skel_static

In [35]:
nodes_position = skel_static.get_nodes_position((4,4,40))



AttributeError: 'Skeleton' object has no attribute 'get_nodes_position'

In [ ]:
node_segment_lut, mask_segment_id = compute_segment_lut(gt_seg, nodes_position)